# Import packages

In [1]:
!pip install torcheval onnx wandb kaggle --quiet

In [2]:
import os
import json
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler
import torch.optim as optim
from torcheval.metrics.functional import multiclass_f1_score
import wandb

# Download data if not found
### *You should upload kaggle.json to the current working dir before running this cell*

In [3]:
if not os.path.exists("Data"):
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !ls ~/.kaggle
    !chmod 600 ~/.kaggle/kaggle.json  # set permission
    !kaggle competitions download -c cassava-leaf-disease-classification -p ./
    !mkdir Data
    !unzip -q -n cassava-leaf-disease-classification.zip -d ./Data
    !rm cassava-leaf-disease-classification.zip

# Wandb Setup
### *Change the env below to be your notebook path, also follow the prompt to enter your wandb token*

In [4]:
os.environ['WANDB_NOTEBOOK_NAME'] = "LinearRegression.ipynb"
wandb.login()

wandb: Currently logged in as: tonghuovo (cassava). Use `wandb login --relogin` to force relogin


True

# *Define Customized Dataset*

In [5]:
class LeafDataset(Dataset):
    def __init__(self, root_path, transform=None):
        self.image_path = root_path + '/train_images'
        self.labels = pd.read_csv(root_path + '/train.csv')
        self.transform = transform

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_path, self.labels['image_id'][idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        if "label" in self.labels.columns:
            label = self.labels['label'][idx]
            sample = (image, label)
        else:
            sample = (image)
        return sample

# Define Torch Module
### *Change this to your own model*

In [6]:
class SoftmaxRegression(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super().__init__()
        # Create the layers of CNN
        self.linear = nn.Linear(n_inputs, n_outputs)

    def forward(self, x):
        # Perform the forward pass through the layers
        x = torch.flatten(x, start_dim=1)
        x = self.linear(x)
        return x

# Define Run Segments

## The Whole Pipeline

In [7]:
def model_pipeline(hyperparameters):
    # tell wandb to get started
    with wandb.init(project="cassava-leaf", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # make the model, data, and optimization problem
        model, train_loader, eval_loader, test_loader, criterion, optimizer = make(config)
        print(model)

        # and use them to train the model
        train(model, train_loader, eval_loader, criterion, optimizer, config)

        # and test its final performance
        test(model, test_loader)

    return model

## 1. Make (Model, loaders...)

### Helper functions for Make

In [8]:
def get_datasets(split, transform):
    total_dataset = LeafDataset('./Data', transform=transform)
    subsets = random_split(total_dataset,
                           split,
                           generator=torch.Generator().manual_seed(42))
    return subsets


def make_loaders(datasets, batch_size, num_workers, balance=False):
    if balance:
        # compute class weights:
        class_weights = [0] * 5
        for _, label in datasets[0]:
            class_weights[label] += 1
        class_weights = [10000 / i for i in class_weights]

        # compute sample weights
        sample_weights = [0] * len(datasets[0])
        for idx, (data, label) in enumerate(datasets[0]):
            sample_weights[idx] = class_weights[label]
        # init weighted sampler
        sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

        train_dataloader = DataLoader(datasets[0], batch_size=batch_size, num_workers=num_workers, sampler=sampler)
    else:
        train_dataloader = DataLoader(datasets[0], batch_size=batch_size, shuffle=True, num_workers=num_workers)
    eval_dataloader = DataLoader(datasets[1], batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_dataloader = DataLoader(datasets[2], batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_dataloader, eval_dataloader, test_dataloader

### Make main function (Adjust Transforms / Model Declaration Here)

In [9]:
def make(config):
    trans_list = [
        transforms.Resize((config.resize, config.resize))
    ]
    if config.flip is not None:
        trans_list.append(transforms.RandomHorizontalFlip(0.3))
        trans_list.append(transforms.RandomVerticalFlip(0.3))
    trans_list.append(transforms.ToTensor())
    trans_list.append(transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]))
    transform = transforms.Compose(trans_list)

    # Make the data
    datasets = get_datasets(config.split, transform)
    train_loader, eval_loader, test_loader = make_loaders(datasets,
                                                          batch_size=config.batch_size,
                                                          num_workers=config.num_workers,
                                                          balance=config.balance)

    # Make the model
    model = SoftmaxRegression(3 * config.resize * config.resize, 5)
    model.cuda()

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(),
                            lr=config.learning_rate,
                            weight_decay=config.weight_decay)
    return model, train_loader, eval_loader, test_loader, criterion, optimizer

## 2. Train

### Helper functions for Train

In [10]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.
    correct, total = 0, 0
    for _, (images, labels) in enumerate(train_loader):
        # move data
        images = images.cuda()
        labels = labels.cuda()

        # forward prop
        output = model(images)
        loss = criterion(output, labels)

        # backward prop
        optimizer.zero_grad()
        loss.backward()

        # update weights
        optimizer.step()

        # record performance
        running_loss += loss.item()
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_loss = running_loss / len(train_loader)
    train_acc = correct / total
    return train_loss, train_acc


def eval_epoch(model, eval_loader, criterion):
    model.eval()
    running_loss = 0.
    correct, total = 0, 0
    for _, (images, labels) in enumerate(eval_loader):
        # move data
        images = images.cuda()
        labels = labels.cuda()

        # forward prop
        output = model(images)
        loss = criterion(output, labels)

        # record performance
        running_loss += loss.item()
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    eval_loss = running_loss / len(eval_loader)
    eval_acc = correct / total
    return eval_loss, eval_acc

### Train main function

In [11]:
def train(model, train_loader, eval_loader, criterion, optimizer, config):
    wandb.watch(model, criterion, log="all", log_freq=10)
    for epoch in range(config.epochs):

        # train model
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
        # evaluate model
        eval_loss, eval_acc = eval_epoch(model, eval_loader, criterion)

        # log to wandb
        wandb.log({"train_loss": train_loss,
                   "train_acc": train_acc,
                   "eval_loss": eval_loss,
                   "eval_acc": eval_acc}, step=epoch)
        print(f"Epoch {epoch}: Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f}, Eval Loss: {eval_loss:.3f}, Eval Acc: {eval_acc:.3f}")

## 3. Test

In [12]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        label_total = torch.tensor([])
        pred_total = torch.tensor([])
        for images, labels in test_loader:
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            # compute correct / total samples
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # concate labels
            label_total = torch.cat((label_total, labels.detach().cpu()), dim=0)
            pred_total = torch.cat((pred_total, predicted.detach().cpu()), dim=0)
        # compute accuracy
        acc = correct / total
        wandb.log({"test_accuracy": acc})
        # compute f1 scores
        pred_total = pred_total.to(torch.int64)
        label_total = label_total.to(torch.int64)
        f1_micro = multiclass_f1_score(pred_total,
                                       label_total,
                                       num_classes=5,
                                       average="micro")
        f1_macro = multiclass_f1_score(pred_total,
                                       label_total,
                                       num_classes=5,
                                       average="macro")
        f1_each = multiclass_f1_score(pred_total,
                                      label_total,
                                      num_classes=5,
                                      average=None)
        print(f"Test Acc: {acc:.3f}, F1 micro: {f1_micro:.3f}, F1 macro: {f1_macro:.3f}, F1 each: {f1_each}")
        f1 = [[f"f1_class_{idx}", value] for idx, value in enumerate(f1_each)]
        f1.append(["f1_micro", f1_micro])
        f1.append(["f1_macro", f1_macro])
        table = wandb.Table(data=f1, columns=["class", "f1_score"])
        wandb.log({"my_bar_chart_1": wandb.plot.bar(table, "class", "f1_score", title="F1 Score")})

        torch.save(model.state_dict(), "model_state.pth")
        wandb.save("model_state.pth")
        torch.onnx.export(model, images, "model.onnx")
        wandb.save("model.onnx")

# Run Pipeline
# Define HyperParameters
### *Change this according to your run, add config if needed*

In [13]:
config = dict(
    model="LinearRegression",
    split=[0.4, 0.2, 0.4],  # train / val / test split ratio
    batch_size=256,
    resize=64,
    num_workers=2,  # number of workers per dataloader
    balance=True,  # weight balance train dataset
    epochs=10,
    learning_rate=0.01,
    weight_decay=0.5,  # L2 regularization hyperparamter for Adam Optimizer
    flip=0.3,
)

for flip in [0.3, None]:
    config['flip'] = flip
    for weight_decay in [0.01, 0.1, 0.5]:
        config['weight_decay'] = weight_decay
        for resize in [64, 128, 224]:
            config['resize'] = resize
            try:
                model = model_pipeline(config)
            except Exception as e:
                print(f"failed run on config: {config}")

SoftmaxRegression(
  (linear): Linear(in_features=12288, out_features=5, bias=True)
)
Epoch 0: Train Loss: 9.053, Train Acc: 0.250, Eval Loss: 4.908, Eval Acc: 0.184
Epoch 1: Train Loss: 3.444, Train Acc: 0.293, Eval Loss: 4.567, Eval Acc: 0.209
Epoch 2: Train Loss: 3.094, Train Acc: 0.302, Eval Loss: 3.176, Eval Acc: 0.259
Epoch 3: Train Loss: 2.504, Train Acc: 0.329, Eval Loss: 3.406, Eval Acc: 0.178
Epoch 4: Train Loss: 2.531, Train Acc: 0.334, Eval Loss: 2.771, Eval Acc: 0.222
Epoch 5: Train Loss: 2.519, Train Acc: 0.345, Eval Loss: 1.913, Eval Acc: 0.349
Epoch 6: Train Loss: 2.692, Train Acc: 0.335, Eval Loss: 2.237, Eval Acc: 0.449
Epoch 7: Train Loss: 2.567, Train Acc: 0.350, Eval Loss: 2.397, Eval Acc: 0.361
Epoch 8: Train Loss: 3.084, Train Acc: 0.322, Eval Loss: 2.814, Eval Acc: 0.273
Epoch 9: Train Loss: 2.461, Train Acc: 0.347, Eval Loss: 2.459, Eval Acc: 0.384
Test Acc: 0.387, F1 micro: 0.387, F1 macro: 0.218, F1 each: tensor([0.0996, 0.2050, 0.1238, 0.5731, 0.0873])


eval_acc,▁▂▃▁▂▅█▆▃▆
eval_loss,█▇▄▄▃▁▂▂▃▂
test_accuracy,▁
train_acc,▁▄▅▇▇█▇█▆█
train_loss,█▂▂▁▁▁▁▁▂▁
eval_acc,0.38435
eval_loss,2.45902
test_accuracy,0.38689
train_acc,0.34654
train_loss,2.46137


SoftmaxRegression(
  (linear): Linear(in_features=12288, out_features=5, bias=True)
)
Epoch 0: Train Loss: 13.815, Train Acc: 0.254, Eval Loss: 5.737, Eval Acc: 0.176
Epoch 1: Train Loss: 2.946, Train Acc: 0.384, Eval Loss: 2.364, Eval Acc: 0.317
Epoch 2: Train Loss: 1.857, Train Acc: 0.451, Eval Loss: 2.506, Eval Acc: 0.270
Epoch 3: Train Loss: 1.630, Train Acc: 0.493, Eval Loss: 2.822, Eval Acc: 0.259
Epoch 4: Train Loss: 1.664, Train Acc: 0.520, Eval Loss: 2.429, Eval Acc: 0.285
Epoch 5: Train Loss: 2.099, Train Acc: 0.462, Eval Loss: 2.501, Eval Acc: 0.438
Epoch 6: Train Loss: 1.835, Train Acc: 0.502, Eval Loss: 3.417, Eval Acc: 0.188
Epoch 7: Train Loss: 1.381, Train Acc: 0.562, Eval Loss: 2.776, Eval Acc: 0.238
Epoch 8: Train Loss: 1.787, Train Acc: 0.511, Eval Loss: 3.547, Eval Acc: 0.191
Epoch 9: Train Loss: 1.366, Train Acc: 0.563, Eval Loss: 2.960, Eval Acc: 0.193
Test Acc: 0.188, F1 micro: 0.188, F1 macro: 0.167, F1 each: tensor([0.0969, 0.1783, 0.1523, 0.2155, 0.1927])


eval_acc,▁▅▃▃▄█▁▃▁▁
eval_loss,█▁▁▂▁▁▃▂▃▂
test_accuracy,▁
train_acc,▁▄▅▆▇▆▇█▇█
train_loss,█▂▁▁▁▁▁▁▁▁
eval_acc,0.19346
eval_loss,2.96005
test_accuracy,0.18754
train_acc,0.56257
train_loss,1.36585
